In [1]:
import pandas as pd
data_in = pd.read_csv('TFData.csv')
clean_data = data_in.fillna(method= 'ffill')


#clean_data['Department'].value_counts()
total_vols = clean_data['Customer ID'].count()

# SSS calculation:

CL = .95   #Confidence Level
MoE = .05  #Margin of Error
uplift = .1

sample_size_numerator = (1.96*1.96) * CL * total_vols * (1-CL)
sample_size_denominator = ((MoE * MoE) * total_vols) + ((1.96*1.96) * CL * (1-CL))
SS = int(sample_size_numerator / sample_size_denominator) 

total_sample_size = SS + (SS * uplift)
os_size = int(total_sample_size/10)


# Calculate sample size by department:

corporate_finance_sample_vols = int((((clean_data.query("Department == 'Corporate Finance'")["Customer ID"].count())/ 
                               total_vols) * total_sample_size).round())

commercial_finance_sample_vols = int((((clean_data.query("Department == 'Commercial Finance'")["Customer ID"].count())/ 
                               total_vols) * total_sample_size).round())

personal_finance_sample_vols = int((((clean_data.query("Department == 'Personal Finance'")["Customer ID"].count())/ 
                               total_vols) * total_sample_size).round())

sample_vols_totals = corporate_finance_sample_vols + commercial_finance_sample_vols + personal_finance_sample_vols


# Sample by Department:

corporate_finance_sample = pd.DataFrame(clean_data[clean_data["Department"] 
                                                   == "Corporate Finance"]).sample(n = corporate_finance_sample_vols)

commercial_finance_sample = pd.DataFrame(clean_data[clean_data["Department"] 
                                                    == "Commercial Finance"]).sample(n = commercial_finance_sample_vols)

personal_finance_sample = pd.DataFrame(clean_data[clean_data["Department"] 
                                                  == "Personal Finance"]).sample(n = personal_finance_sample_vols)


# Merge Dataframes:

sample_output = pd.concat([corporate_finance_sample, commercial_finance_sample, personal_finance_sample])


#Over sample DF for additional sample for any rejects

oversample_df = pd.concat([clean_data, sample_output]).drop_duplicates(keep = False)


# Get additional cases to cover rejects:

corporate_finance_os = pd.DataFrame(oversample_df[oversample_df["Department"] 
                                                   == "Corporate Finance"]).sample(n = os_size)

commercial_finance_os = pd.DataFrame(oversample_df[oversample_df["Department"] 
                                                    == "Commercial Finance"]).sample(n = os_size)

personal_finance_os = pd.DataFrame(oversample_df[oversample_df["Department"] 
                                                  == "Personal Finance"]).sample(n = os_size)

# Merge over sample DataFrames:

os_output = pd.concat([corporate_finance_os, commercial_finance_os, personal_finance_os])

# Save workgroup volumes for Summary:

corporate_finance_vols = clean_data.query("Department == 'Corporate Finance'")["Customer ID"].count()
commercial_finance_vols = clean_data.query("Department == 'Commercial Finance'")["Customer ID"].count()
personal_finance_vols = clean_data.query("Department == 'Personal Finance'")["Customer ID"].count()

volume_totals = corporate_finance_vols + commercial_finance_vols + personal_finance_vols

# Create summary of the sampling volumes:

summary_dict = {
    'Workgroup' : ["Corporate Finance", "Commercial Finance", "Personal Finance", "Totals"],
    'Overall Volumes' : [corporate_finance_vols, commercial_finance_vols, personal_finance_vols, volume_totals],
    'Sample Volumes' : [corporate_finance_sample_vols, commercial_finance_sample_vols, personal_finance_sample_vols, sample_vols_totals],
}

sampling_summary = pd.DataFrame(summary_dict)

# Create a Pandas Excel Writer using XlsxWriter as the engine:

writer = pd.ExcelWriter('QA Sampling.xlsx', engine='xlsxwriter')

# Write sample and oversample df's as seperate tabs in workbook:

sampling_summary.to_excel(writer, sheet_name='Sampling Summary')
sample_output.to_excel(writer, sheet_name='Sample Data')
os_output.to_excel(writer, sheet_name='Oversample Data')

# Close the Excel Writer and output te Excel file:
writer.save()

# Calculate sampling %

sampling_pct = (sample_vols_totals / volume_totals) * 100


# Print a Summary:

print('\033[1m' + "NEW BUSINESS VOLUMES AND SAMPLING SUMMARY\n" + '\033[0m')
print("Below is a summary of the QC sampling volumes for new business received in 2020.")
print("New business volumes for 2020 are", total_vols, "advances of which",'{:.0f}%'.format(sampling_pct),
      "(",sample_vols_totals, ") have been sampled for QC testing as follows:\n" )
print(sampling_summary)

NEW BUSINESS VOLUMES AND SAMPLING SUMMARY

Below is a summary of the QC sampling volumes for new business received in 2020.
New business volumes for 2020 are 620 advances of which 11% ( 71 ) have been sampled for QC testing as follows:

            Workgroup  Overall Volumes  Sample Volumes
0   Corporate Finance              224              26
1  Commercial Finance              193              22
2    Personal Finance              203              23
3              Totals              620              71
